## An RNN classifier for rumoreval that solely uses the text attribute of the tweets for classification. Similar to homework 6. 

In [14]:
import pandas as pd
import sys
sys.path.insert(1, "./code/rnn_text")
import train
import model
import helpers
import sys
sys.path.insert(1, "./code/feature-extraction/embed-extractor")
sys.path.insert(1, "./code/feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./code/feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
import classifiers
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
import helpers
from torch.autograd import Variable


The only attribute here is the text!

In [15]:
attributes = ['text']

Setting our parameters. RNN initialization extremely similar to the one used in the PyTorch RNN tutorial for classifying baby names.

In [16]:
import string
n_epochs = 2000
hidden_size = 20
learning_rate = 0.001


decoder = model.RNN(len(helpers.all_letters), hidden_size=hidden_size, output_size =3)
decoder_optimizer = torch.optim.SGD(decoder.parameters(), learning_rate)

## From here we can define functions that normalize and transform our data suitable for the classifier.

In [17]:
# need stuff from main.py to test transformation into Tensors

def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("code/feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    #word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells
    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 2
    df.loc[df.classification == 'unverified', 'classification'] = 0
    
    # getting the labels
    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()

    
    values = np.array(values)
    if index == 1:
        return df.index, values

    return labels, values

## From here, you can set where your training and target data comes from (we just "call it" dev_values here). Please maintain the index variable seen in the second line.

In [18]:
tr_labels, tr_values = labels_and_vectors('output/simple/train_data_simple.pickle')
indices, dev_values,= labels_and_vectors('output/simple/goldtest_data_simple.pickle', index=1)

FileNotFoundError: [Errno 2] No such file or directory: './feature-extraction/twitter-parser/tweets.txt'

Further transformation of data such that we can make it suitable for the torch library.

In [ ]:
tvs = [arr.tolist() for arr in tr_values]
dvs = [arr.tolist() for arr in dev_values]
tls = [l.item() for l in tr_labels]

An example...

In [ ]:
tvs[0]

Setting our parameters. RNN initialization extremely similar to the one used in the PyTorch RNN tutorial for classifying baby names.

In [ ]:
import string
n_epochs = 2000
hidden_size = 20
learning_rate = 0.001
decoder = model.RNN(len(helpers.all_letters), hidden_size=hidden_size, output_size =3)
decoder_optimizer = torch.optim.SGD(decoder.parameters(), learning_rate)


Similar to the pytorch tutorial for RNNs, this is a function that goes through the training data.

In [ ]:
def train_all():
    for i, label in enumerate(tls):
        tensor = helpers.line_to_tensor(tvs[i][0])  
        category_tensor = Variable(torch.LongTensor([label]))
        line_tensor = Variable(tensor)
        output, loss = train.train(category_tensor, line_tensor,decoder)
        decoder_optimizer.step()

In [ ]:
By executing this script, we train the model get the label predictions.

In [ ]:
for i in range(n_epochs):
    train_all()
    
predictions = []
for d in dvs:
    tensor = helpers.line_to_tensor(d)  
    line_tensor = Variable(tensor)
    output = decoder.predict(line_tensor)
    predictions.append(output)

    
    
ps = []

for i, p in enumerate(predictions):
    p = p[0][0]
    if p == 2:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 0:
        ps.append('unverified')

pred_dict = {index:(ps[i],1) for i,index in enumerate(indices)}

## Output to json and scoring script. The first argument is the gold set and the second argument is the predictions you should have generated.



In [ ]:
with open('output/rnn/goldtest_rnn_text.json', 'w') as outfile:
    json.dump(pred_dict, outfile)

In [ ]:
!python3 scorer/score.py data/semeval2017-task8-dataset/goldtest/subtaskb.json output/rnn/goldtest_rnn_text.json